In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [2]:
use_GPU = torch.cuda.is_available()
if use_GPU:
    mode = {"name": "cuda", "device": torch.device("cuda")}
else:
    mode = {"name": "cpu", "device": torch.device("cpu")}
error_criterion = nn.MSELoss().to(mode["device"])
loss_criterion = nn.MSELoss().to(mode["device"])

in_dim = 1
hidden_dim = 64
out_dim = 1
sequence_length = 24
batch_size = 128

num_epochs=15
num_workers=11
lr = 1e-3
regularization=1e-6

input = torch.randn(batch_size, sequence_length)
random_data_y = torch.randn(batch_size, 1)
print(input.shape, random_data_y.shape)

torch.Size([128, 24]) torch.Size([128, 1])


In [3]:
class FCN(nn.Module):
    
    def __init__(self, in_dim, hidden_dim, out_dim, sequence_length):

        super(FCN, self).__init__()
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim
        self.sequence_length = sequence_length

        self.layer_in = nn.Linear(sequence_length, hidden_dim,bias=False)
        self.fcn = nn.Linear(hidden_dim, hidden_dim)
        self.layer_out = nn.Linear(hidden_dim, out_dim, bias=False)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward (self,input):
        out = self.layer_in(input)
        out = self.sigmoid(out)
        out = self.fcn(out)
        out = self.sigmoid(out)
        out = self.layer_out(out)
      
        return out

In [4]:
class LSTM(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(LSTM, self).__init__()
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim
        self.sequence_length = sequence_length
        # lstm1, lstm2, linear are all layers in the network
        self.lstm1 = nn.LSTMCell(in_dim, hidden_dim)
        self.lstm2 = nn.LSTMCell(hidden_dim, hidden_dim)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.linear_out = nn.Linear(hidden_dim*sequence_length, out_dim)

        
    def forward(self, y):
        outputs = []
        h_t = torch.zeros(y.size(0), self.hidden_dim, dtype=torch.float32)
        c_t = torch.zeros(y.size(0), self.hidden_dim, dtype=torch.float32)
        h_t2 = torch.zeros(y.size(0), self.hidden_dim, dtype=torch.float32)
        c_t2 = torch.zeros(y.size(0), self.hidden_dim, dtype=torch.float32)
        
        for time_step in y.split(1, dim=1):
            # N, 1
            h_t, c_t = self.lstm1(time_step, (h_t, c_t)) # initial hidden and cell states
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2)) # new hidden and cell states
            output = self.linear(h_t2) # output from the last FC layer
            outputs.append(output)
        # transform list to tensor    
        outputs = torch.cat(outputs, dim=1)
        out = self.linear_out(outputs)
        return out

In [5]:
class LSTM2(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(LSTM2, self).__init__()
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim

        self.lstm1 = nn.LSTMCell(in_dim, hidden_dim)
        self.lstm2 = nn.LSTMCell(hidden_dim, hidden_dim)

        self.T_A = nn.Linear(sequence_length*hidden_dim, sequence_length)
        
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.linear_out = nn.Linear(hidden_dim, out_dim)

        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
    def forward(self, y):
        outputs = []
        h_t = torch.zeros(y.size(0), self.hidden_dim, dtype=torch.float32)
        c_t = torch.zeros(y.size(0), self.hidden_dim, dtype=torch.float32)
        h_t2 = torch.zeros(y.size(0), self.hidden_dim, dtype=torch.float32)
        c_t2 = torch.zeros(y.size(0), self.hidden_dim, dtype=torch.float32)
        
        for time_step in y.split(1, dim=1):
            h_t, c_t = self.lstm1(time_step, (h_t, c_t)) # initial hidden and cell states
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2)) # new hidden and cell states
            output = self.linear(h_t2) # output from the last FC layer
            outputs.append(output)
            
        total_ht = outputs[0]
        for i in range(1, len(outputs)):
            total_ht = torch.cat((total_ht, outputs[i]), 1)

        beta_t =  self.relu(self.T_A(total_ht))
        beta_t = self.softmax(beta_t)

        out = torch.zeros(y.size(0), self.hidden_dim)

        for i in range(len(outputs)):
                      
            out = out + outputs[i]*beta_t[:,i].reshape(out.size(0), 1)

        out = self.linear_out(out)
        
        return out

In [6]:
model = FCN(in_dim, hidden_dim, out_dim, sequence_length)
y_pred = model(input)
print(y_pred.shape)
print(random_data_y.shape)
mse_loss = nn.MSELoss()
mse_loss(y_pred, random_data_y)

torch.Size([128, 1])
torch.Size([128, 1])


tensor(1.0261, grad_fn=<MseLossBackward0>)

In [7]:
model = LSTM(1, 64, 1)
y_pred = model(input)
print(y_pred.shape)
print(random_data_y.shape)
mse_loss = nn.MSELoss()
mse_loss(y_pred, random_data_y)

torch.Size([128, 1])
torch.Size([128, 1])


tensor(1.0224, grad_fn=<MseLossBackward0>)

In [8]:
model = LSTM2(1, 64, 1)
y_pred = model(input)
print(y_pred.shape)
print(random_data_y.shape)
mse_loss = nn.MSELoss()
mse_loss(y_pred, random_data_y)

torch.Size([128, 1])
torch.Size([128, 1])


tensor(1.0591, grad_fn=<MseLossBackward0>)